#第1章 统计学习方法概论


高斯于1823年在误差$e_1,..., e_n$独立同分布的假定下,证明了最小二乘方法的一个最优性质: 
在所有无偏的线性估计类中,最小二乘方法((Least-square fitting))是其中方差最小的

## 使用最小二乘法拟和曲线
对于数据$(x_i, y_i)(i=1, 2, 3...,m)$

拟合出函数$h(x)$

有误差，即残差：$r_i=h(x_i)-y_i$

此时L2范数(残差平方和)最小时，h(x) 和 y 相似度最高，更拟合

一般的H(x)为n次的多项式，$H(x)=w_0+w_1x+w_2x^2+...w_nx^n$

$w(w_0,w_1,w_2,...,w_n)$为参数

最小二乘法就是要找到一组 $w(w_0,w_1,w_2,...,w_n)$ 使得$\sum_{i=1}^n(h(x_i)-y_i)^2$ (残差平方和) 最小

即，求 $min\sum_{i=1}^n(h(x_i)-y_i)^2$



举例：我们用目标函数$y=sin2{\pi}x$, 加上一个正太分布的噪音干扰，用多项式去拟合【例1.1 11页】

In [75]:
import numpy as np
import scipy as sp
from scipy.optimize import leastsq
import matplotlib.pyplot as plt

ps: numpy.poly1d([1,2,3]) 生成  $1x^2+2x^1+3x^0$

In [107]:
def real_func(x):
    y = np.sin(2*np.pi*x)
    return y

def poly_func(p, x):
    f = np.poly1d(p)
    return f(x)

def compute_residuals(p, x, y):
    ret = poly_func(p, x) - y
    return ret

使用最小二乘法拟合残差的平方和,

**Scipy最优化函数库Optimization**的leastsq()函数传入误差计算函数和初始值，该初始值将作为误差计算函数的第一个参数传入；

计算的结果r是一个包含两个元素的元组，第一个元素是一个数组，表示拟合后的参数k、b；第二个元素如果等于1、2、3、4中的其中一个整数

In [110]:
# 开始构建数据集
x_points = np.linspace(0, 1, 1000)
x = np.linspace(0, 1, 10)
y = real_func(x)
def fit(m, x, y, x_points):    
    '''
    使用最小二乘法拟合曲线
    :param x: array like 
    :param m: 多项式的次数
    :return: 
    '''
    # 多项式参数初始化
    p_init = np.random.rand(m + 1)
    p_lsq = leastsq(compute_residuals, p_init, args=(x, y))
    print('Fitting Parameters:', p_lsq[0])
    
    plt.figure(figsize=(10, 12))
    plt.plot(x_points, real_func(x_points), label='real')
    plt.plot(x_points, poly_func(p_lsq[0], x_points), label='poly')
    plt.scatter(x, y, label='noice point', color='green')
    plt.title('m='+str(m))
    plt.legend()
    plt.show()


In [111]:
for m in [0,1,3,5,7,9]:
    fit(m, x, y, x_points)

Fitting Parameters: [7.18347125e-10]
Fitting Parameters: [-1.34876165  0.67438082]

Fitting Parameters: [ 21.07651922 -31.61477885  10.64664007  -0.05419022]


Fitting Parameters: [-5.39644498e+01  1.34911125e+02 -9.66237211e+01  1.00244571e+01
  5.65130257e+00  6.43347585e-04]


Fitting Parameters: [ 5.82656880e+01 -2.03929908e+02  2.27044378e+02 -5.77861758e+01
 -2.84740691e+01 -1.46767424e+00  6.34776234e+00 -6.74347220e-07]


Fitting Parameters: [-3.50069823e+01  1.57531420e+02 -2.39484331e+02  1.03048530e+02
  3.95696989e+01  1.10277430e+01 -4.31219765e+01  1.58539686e-01
  6.27735815e+00  1.87419450e-23]


In [112]:
y_noice = [np.random.normal(0, 0.1)+y_ for y_ in y]
for m in [0,1,3,5,7,9]:
    fit(m, x, y_noice, x_points)


Fitting Parameters: [0.02456806]
Fitting Parameters: [-1.29794725  0.67354169]


Fitting Parameters: [ 2.11497260e+01 -3.13514661e+01  1.03659962e+01 -2.28248934e-03]


Fitting Parameters: [-5.06127061e+01  1.29608974e+02 -9.53945445e+01  1.15385612e+01
  4.92111564e+00  6.94067928e-02]


Fitting Parameters: [-3.48471861e+02  1.21223879e+03 -1.70901491e+03  1.25708088e+03
 -4.90042857e+02  7.72642584e+01  9.99403250e-01  7.23455113e-02]


Fitting Parameters: [ 1.48042822e+04 -6.85006993e+04  1.33094273e+05 -1.40899435e+05
  8.81090955e+04 -3.29047293e+04  7.04598571e+03 -7.88310395e+02
  3.95924196e+01  7.08280244e-02]


# 将L2正则化加入到高阶多项式回归中

结果显示过拟合， 引入正则化项(regularizer)，降低过拟合

$Q(x)=\sum_{i=1}^n(h(x_i)-y_i)^2+\lambda||w||^2$。

回归问题中，损失函数是平方损失，正则化可以是参数向量的L2范数,也可以是L1范数。

L1: regularization*abs(p)

L2: 0.5 * regularization * np.square(p)

In [132]:
def residuals_func_regularization(p, x, y, alpha=0.1):
    ret = poly_func(p, x) - y 
    ret = np.append(ret, np.sqrt(1/(2*len(p))*alpha * np.square(p)))
    return ret

def fit_regularization(m, x, y, x_points, alpha):    
    '''
    使用最小二乘法拟合曲线
    :param x: array like 
    :param m: 多项式的次数
    :return: 
    '''
    # 多项式参数初始化
    p_init = np.random.rand(m + 1)
    p_lsq = leastsq(residuals_func_regularization, p_init, args=(x, y, alpha))
    print('Fitting Parameters:', p_lsq[0])
    
    plt.figure(figsize=(10, 12))
    plt.plot(x_points, real_func(x_points), label='real')
    plt.plot(x_points, poly_func(p_lsq[0], x_points), label='poly')
    plt.scatter(x, y, label='noice point', color='green')
    plt.title('m='+str(m))
    plt.legend()
    plt.show()

In [133]:
for m in [0, 1, 3, 9, 12, 15, 19]:
    fit_regularization(m, x, y_noice, x_points, 0.1)

Fitting Parameters: [0.02444584]
Fitting Parameters: [-1.22241034  0.63261018]


Fitting Parameters: [ 1.10395378 -1.22493213 -1.03364428  0.66241891]


Fitting Parameters: [ 0.98313027  0.88561529  0.71461908  0.43825198  0.01823935 -0.57376458
 -1.28522918 -1.70649574  0.00749243  0.6190283 ]
Fitting Parameters: [ 0.37717464  0.44044341  0.48952821  0.51360054  0.49657947  0.41505483
  0.23631337 -0.08085127 -0.57211248 -1.2014105  -1.59411658  0.0279286
  0.6070138 ]


Fitting Parameters: [ 0.03983842  0.11950351  0.20086069  0.28097488  0.35521194  0.4164473
  0.45387308  0.45135358  0.38527487  0.222456   -0.07925637 -0.55733631
 -1.17903874 -1.57543323  0.02429724  0.60542162]


Fitting Parameters: [-0.15745361 -0.09971909 -0.03715452  0.02999787  0.10119693  0.175359
  0.25065713  0.32413287  0.39115548  0.44462622  0.47382231  0.46277008
  0.38815154  0.21728322 -0.09116426 -0.57337223 -1.1946218  -1.58352548
  0.03108011  0.60580951]
